In [22]:
import argparse
import easydict
from trainer import Trainer
from utils import init_logger, load_tokenizer, set_seed, MODEL_CLASSES, MODEL_PATH_MAP
from data_loader import load_and_cache_examples
import main

In [20]:
args = easydict.EasyDict({
    "input_file":"sample_pred_in.txt",
    "output_file":"sample_pred_out.txt",
    "model_dir":"./model",
    "batch_size":32,
    "no_cuda":store_true
})
args.model_name_or_path = MODEL_PATH_MAP[args.model_type]

In [25]:
args

{'task': 'naver-ner',
 'model_dir': './model',
 'data_dir': './data',
 'pred_dir': './preds',
 'train_file': 'train.tsv',
 'test_file': 'test.tsv',
 'label_file': 'label.txt',
 'write_pred': 'store_true',
 'model_type': 'kobert',
 'seed': 42,
 'train_batch_size': 32,
 'eval_batch_size': 32,
 'max_seq_len': 50,
 'learning_rate': 5e-05,
 'num_train_epochs': 20.0,
 'weight_decay': 0.0,
 'gradient_accumulation_steps': 1,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'max_steps': -1,
 'warmup_steps': 0,
 'logging_steps': 1000,
 'save_steps': 1000,
 'do_train': 'store_true',
 'do_eval': 'store_true',
 'no_cuda': 'store_true',
 'model_name_or_path': 'monologg/kobert'}

## main

In [23]:
init_logger()
set_seed(args)

In [24]:
tokenizer = load_tokenizer(args)

In [27]:
tokenizer

PreTrainedTokenizer(name_or_path='monologg/kobert', vocab_size=8002, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})